## notebook to interact with google cloud bigquery tables

In [43]:
from google.cloud import storage, bigquery
import google.auth
import pandas as pd

from shapely.geometry import Polygon
from pyproj import Transformer
import requests

# Get default credentials
credentials, project_id = google.auth.default()

# Initialize clients
bq_client = bigquery.Client(credentials=credentials, project=project_id)
storage_client = storage.Client(credentials=credentials, project=project_id)

## read tables

In [44]:
# List all datasets in the project
print(f"Project ID: {project_id}")
print("\nDatasets in project:")
datasets = list(bq_client.list_datasets())

if datasets:
    for dataset in datasets:
        print(f"  - {dataset.dataset_id}")
        
        # List tables in each dataset
        dataset_ref = bq_client.dataset(dataset.dataset_id)
        tables = list(bq_client.list_tables(dataset_ref))
        
        if tables:
            print(f"{len(tables)} Tables in {dataset.dataset_id}:")
            for table in tables:
                print(f"      - {table.table_id}")
        else:
            print(f"    No tables in {dataset.dataset_id}")
        print()
else:
    print("No datasets found in this project.")

Project ID: useful-theory-442820-q8

Datasets in project:
  - shoreline_metadata
2 Tables in shoreline_metadata:
      - islands
      - shoreline_data



## functions

In [52]:
def load_geojson_from_url(url):
    """Load GeoJSON data from a public URL."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        st.warning(f"Could not load GeoJSON from {url}: {e}")
        return None
    
def _calculate_shoreline_area(coordinates) -> float:
    """Calculate approximate area of shoreline square meters."""
    total_area = 0.0
    try:
        from shapely.geometry import Polygon
        from pyproj import Transformer
        
        # Convert coordinates to a projected system for area calculation
        transformer = Transformer.from_crs("EPSG:4326", "EPSG:32643", always_xy=True)
        projected_coords = [transformer.transform(lon, lat) for lon, lat in coordinates]
        
        # Create polygon and calculate area
        polygon = Polygon(projected_coords)
        total_area = polygon.area  # Area in square meters

        return total_area
    except Exception as e:
        return total_area  

## update tables

In [46]:
# read the shoreline table
for table in tables:
    if table.table_id == "shoreline_data":
        shoreline_table = table
        break
# create a dataframe from shoreline_table
if shoreline_table:
    area_df = bq_client.list_rows(shoreline_table).to_dataframe()
else:
    print("Shoreline table not found.")

In [56]:
# for each row in area_df, update the shoreline_data table with the area value
for index, row in area_df.iterrows():
    # get the geojson from google cloud storage
    geojson = load_geojson_from_url(row['geojson_path'])

    # get the coordinates from the geojson
    coordinates = geojson['features'][0]['geometry']['coordinates']
    # calculate the area in square meters

    area_sq_meters = _calculate_shoreline_area(coordinates)
    
    print(f"Calculated area for row {index}: {area_sq_meters} square meters")

    update_query = f"""
    UPDATE `useful-theory-442820-q8.shoreline_metadata.shoreline_data`
    SET area_enclosed_m2 = {area_sq_meters}
    WHERE site_id = '{row['site_id']}' AND timestamp = '{row['timestamp']}'
    """
    result = bq_client.query(update_query)
    print(f"Updated {result.num_dml_affected_rows} rows for site_id = {row['site_id']} and timestamp = {row['timestamp']}")

Calculated area for row 0: 50221.30690142016 square meters
Updated None rows for site_id = Fenfushi and timestamp = 2017-03-08 05:37:11+00:00
Calculated area for row 1: 49159.66083195888 square meters
Updated None rows for site_id = Fenfushi and timestamp = 2017-03-08 05:37:11+00:00
Calculated area for row 1: 49159.66083195888 square meters
Updated None rows for site_id = Fenfushi and timestamp = 2017-04-07 05:37:11+00:00
Calculated area for row 2: 51673.385066325325 square meters
Updated None rows for site_id = Fenfushi and timestamp = 2017-04-07 05:37:11+00:00
Calculated area for row 2: 51673.385066325325 square meters
Updated None rows for site_id = Fenfushi and timestamp = 2017-04-17 05:37:21+00:00
Calculated area for row 3: 50796.59574612601 square meters
Updated None rows for site_id = Fenfushi and timestamp = 2017-04-17 05:37:21+00:00
Calculated area for row 3: 50796.59574612601 square meters
Updated None rows for site_id = Fenfushi and timestamp = 2017-04-27 05:37:21+00:00
Calc

In [51]:
# read the updated shoreline table
area_df_updated = bq_client.list_rows(shoreline_table).to_dataframe()
print(area_df_updated.head())

    site_id                 timestamp                   date_processed  \
0  Fenfushi 2017-03-08 05:37:11+00:00 2025-10-24 18:32:09.047625+00:00   
1  Fenfushi 2021-03-07 05:37:21+00:00 2025-10-24 18:32:14.108813+00:00   
2  Fenfushi 2019-09-09 05:37:19+00:00 2025-10-24 18:32:12.443998+00:00   
3  Fenfushi 2023-02-20 05:18:39+00:00 2025-10-24 18:32:16.298262+00:00   
4  Fenfushi 2021-04-06 05:37:11+00:00 2025-10-24 18:32:14.292552+00:00   

                                        geojson_path  \
0  https://storage.googleapis.com/littoral-public...   
1  https://storage.googleapis.com/littoral-public...   
2  https://storage.googleapis.com/littoral-public...   
3  https://storage.googleapis.com/littoral-public...   
4  https://storage.googleapis.com/littoral-public...   

                                 simplified_geometry  shoreline_length_m  \
0  LINESTRING(72.9021088839923 5.3809810065243, 7...         1076.378547   
1  LINESTRING(72.9020162678987 5.38090860982317, ...         1123.